# 1. Importing modules and functions

In [3]:
import numpy as np
import pandas as pd
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem, Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors
import chembl_structure_pipeline
from molvs import standardize_smiles
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.model_selection import permutation_test_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_predict
from sklearn import metrics
from sklearn.metrics import pairwise_distances
import joblib
import pickle
from numpy import savetxt
from padelpy import from_sdf
from IPython.display import HTML
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.ensemble import GradientBoostingRegressor
from sklearn_genetic import GASearchCV
from sklearn_genetic.space import Categorical, Integer, Continuous
import warnings
warnings.filterwarnings('ignore')

# 2.Data entry and curation work set

In [4]:
uploaded_file_ws="datasets/HDAC3_work.sdf"
supplier_ws = Chem.ForwardSDMolSupplier(uploaded_file_ws,sanitize=False)
failed_mols_ws = []
all_mols_ws =[]
wrong_structure_ws=[]
wrong_smiles_ws=[]
y_tr = []
y_bad_index=[]

for i, m in enumerate(supplier_ws):
    structure = Chem.Mol(m)
    all_mols_ws.append(structure)
    y_tr.append(m.GetProp("pchembl_value_mean"))
    try:
        Chem.SanitizeMol(structure)
    except:
        failed_mols_ws.append(m)
        wrong_smiles_ws.append(Chem.MolToSmiles(m))
        wrong_structure_ws.append(str(i+1))
        y_bad_index.append(i)
print('Original data: ', len(all_mols_ws), 'molecules')
print('Failed data: ', len(failed_mols_ws), 'molecules')
number_ws =[]
for i in range(len(failed_mols_ws)):
        number_ws.append(str(i+1))
bad_molecules_ws = pd.DataFrame({'No. failed molecule in original set': wrong_structure_ws, 'SMILES of wrong structure: ': wrong_smiles_ws, 'No.': number_ws}, index=None)
bad_molecules_ws = bad_molecules_ws.set_index('No.')
bad_molecules_ws

Original data:  1400 molecules
Failed data:  0 molecules


,No. failed molecule in original set,SMILES of wrong structure:
No.,,


deleting activity values for substances with incorrect structure

In [5]:
y_tr[:] = [x for i,x in enumerate(y_tr) if i not in y_bad_index]

In [6]:
len(y_tr)

1400

# 3.Standardization SDF file for work set

In [7]:
all_mols_ws[:] = [x for i,x in enumerate(all_mols_ws) if i not in y_bad_index] 
records = []
for i in range(len(all_mols_ws)):
    record = Chem.MolToSmiles(all_mols_ws[i])
    records.append(record)

moldf_ws = []
for i,record in enumerate(records):
    standard_record = standardize_smiles(record)
    m = Chem.MolFromSmiles(standard_record)
    moldf_ws.append(m)
    
print('Kept data: ', len(moldf_ws), 'molecules')

Kept data:  1400 molecules


# 4.Data entry and curation test set

In [60]:
uploaded_file_ts="datasets/HDAC3_test.sdf"
supplier_ts = Chem.ForwardSDMolSupplier(uploaded_file_ts,sanitize=False)
failed_mols_ts = []
all_mols_ts =[]
wrong_structure_ts=[]
wrong_smiles_ts=[]
y_ts = []
y_bad_index=[]
for i, m in enumerate(supplier_ts):
    structure = Chem.Mol(m)
    all_mols_ts.append(structure)
    y_ts.append(m.GetProp("pchembl_value_mean"))
    try:
        Chem.SanitizeMol(structure)
    except:
        failed_mols_ts.append(m)
        wrong_smiles_ts.append(Chem.MolToSmiles(m))
        wrong_structure_ts.append(str(i+1))
        y_bad_index.append(i)
print('Original data: ', len(all_mols_ts), 'molecules')
print('Failed data: ', len(failed_mols_ts), 'molecules')
number_ts =[]
for i in range(len(failed_mols_ts)):
        number_ts.append(str(i+1))
bad_molecules_ts = pd.DataFrame({'No. failed molecule in original set': wrong_structure_ts, 'SMILES of wrong structure: ': wrong_smiles_ts, 'No.': number_ts}, index=None)
bad_molecules_ts = bad_molecules_ts.set_index('No.')
bad_molecules_ts

Original data:  351 molecules
Failed data:  0 molecules


,No. failed molecule in original set,SMILES of wrong structure:
No.,,


deleting activity values for substances with incorrect structure

In [61]:
y_ts[:] = [x for i,x in enumerate(y_ts) if i not in y_bad_index]

In [62]:
len(y_ts)

351

# 5.Standardization SDF file for test set

In [11]:
all_mols_ts[:] = [x for i,x in enumerate(all_mols_ts) if i not in y_bad_index] 
records = []
for i in range(len(all_mols_ts)):
    record = Chem.MolToSmiles(all_mols_ts[i])
    records.append(record)

moldf_ts = []
for i,record in enumerate(records):
    standard_record = standardize_smiles(record)
    m = Chem.MolFromSmiles(standard_record)
    moldf_ts.append(m)
    
print('Kept data: ', len(moldf_ts), 'molecules')

Kept data:  351 molecules


# 6.Calculation MorganFingerprint for work set

In [12]:
fp_tr = [AllChem.GetMorganFingerprintAsBitVect(m, radius=2,nBits=1024,useFeatures=False,useChirality = False) for m in moldf_ws]

In [13]:
def rdkit_numpy_convert(fp_tr):
    output = []
    for f in fp_tr:
        arr = np.zeros((1,))
        DataStructs.ConvertToNumpyArray(f, arr)
        output.append(arr)
    return np.asarray(output)

In [14]:
from numpy import savetxt
x_tr = rdkit_numpy_convert(fp_tr)

In [15]:
x_tr.shape

(1400, 1024)

# 7.Calculation MorganFingerprint for test set

In [16]:
fp_ts = [AllChem.GetMorganFingerprintAsBitVect(m, radius=2,nBits=1024,useFeatures=False,useChirality = False) for m in moldf_ts]

In [17]:
def rdkit_numpy_convert(fp_ts):
    output = []
    for f in fp_ts:
        arr = np.zeros((1,))
        DataStructs.ConvertToNumpyArray(f, arr)
        output.append(arr)
    return np.asarray(output)

In [18]:
x_ts = rdkit_numpy_convert(fp_ts)

In [19]:
x_ts.shape

(351, 1024)

In [20]:
type(x_tr)

numpy.ndarray

In [21]:
x_tr

array([[0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]])

In [22]:
x_tr = np.array(x_tr, dtype=np.float32)
y_tr = np.array(y_tr, dtype=np.float32)

In [24]:
y_tr

array([ 4.01,  4.04,  4.05, ..., 10.03, 10.06, 10.1 ], dtype=float32)

 ## GradientBoostingRegressor model building and validation

In [25]:
seed = 42

In [26]:
cv=KFold(n_splits=5, random_state=seed, shuffle=True)

In [34]:
param_grid = {'learning_rate': Continuous(0.01, 0.04),
                  'subsample': Continuous(0.5, 0.9),
                  'n_estimators': Integer(100, 2000),
                  'max_depth': Integer(4, 10)
                 }

In [35]:
evolved_estimator= GASearchCV(estimator=GradientBoostingRegressor(),
                              cv=cv,
                              scoring='r2',
                              param_grid=param_grid,
                              n_jobs=-1,
                              verbose=True,
                              population_size=10,
                              generations=10)

In [36]:
evolved_estimator.fit(x_tr, y_tr)

gen	nevals	fitness 	fitness_std	fitness_max	fitness_min
0  	10    	0.665567	0.0149623  	0.686541   	0.629649   
1  	20    	0.677786	0.00282298 	0.68203    	0.673452   
2  	20    	0.681167	0.00274076 	0.685036   	0.677433   
3  	20    	0.683326	0.00104548 	0.685036   	0.68203    
4  	20    	0.683737	0.000735857	0.685036   	0.68203    
5  	20    	0.683709	0.0032766  	0.685954   	0.674058   
6  	20    	0.685251	0.00111769 	0.688017   	0.684145   
7  	20    	0.683569	0.00278451 	0.688017   	0.679918   
8  	20    	0.685195	0.0022945  	0.688017   	0.680553   
9  	20    	0.686458	0.00224534 	0.688273   	0.680426   
10 	20    	0.683122	0.00798792 	0.688273   	0.661498   
11 	20    	0.685905	0.00350718 	0.688273   	0.675841   
12 	20    	0.687081	0.00131348 	0.688273   	0.684424   
13 	20    	0.686843	0.00146986 	0.688273   	0.684122   
14 	20    	0.687339	0.0019523  	0.690597   	0.683295   
15 	20    	0.687206	0.00264352 	0.690597   	0.680563   
16 	20    	0.688852	0.0015811  	0.690597   	0.68

GASearchCV(cv=KFold(n_splits=5, random_state=42, shuffle=True),
           estimator=GradientBoostingRegressor(learning_rate=0.015369880344468845,
                                               max_depth=10, n_estimators=1394,
                                               subsample=0.6147217030120626),
           generations=20, n_jobs=-1,
           param_grid={'learning_rate': <sklearn_genetic.space.space.Continuous object at 0x00000263AE502A70>,
                       'max_depth': <sklearn_genetic.space.space.Integer object at 0x00000263AE5017E0>,
                       'n_estimators': <sklearn_genetic.space.space.Integer object at 0x00000263AE502F20>,
                       'subsample': <sklearn_genetic.space.space.Continuous object at 0x00000263AE503490>},
           population_size=10, return_train_score=True, scoring='r2',
           warm_start_configs=[])

In [37]:
# Best parameters found
print(evolved_estimator.best_params_)

{'learning_rate': 0.015369880344468845, 'subsample': 0.6147217030120626, 'n_estimators': 1394, 'max_depth': 10}
